# **04. Model Selection**

### Librerias

In [1]:
import sys
import pandas as pd 
import numpy as np
import sklearn
from sklearn.pipeline import Pipeline
from sklearn import metrics


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc, \
                            silhouette_score, recall_score, precision_score, make_scorer, fbeta_score, \
                            roc_auc_score, f1_score, precision_recall_curve, accuracy_score, roc_auc_score, \
                            classification_report, confusion_matrix, calinski_harabasz_score, silhouette_samples

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB, GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
import lightgbm as lgb
from sklearn.svm import SVC, LinearSVC, NuSVC
from xgboost import XGBClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
from joblib import dump, load

warnings.filterwarnings('ignore')

### Funciones

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 10000)
pd.set_option('display.width', 10000)

In [3]:
sys.path.append('../src/')
import functions as f
sys.path.remove('../src/')

In [4]:
# Constants
seed= 12345

### Carga de datos

In [5]:
df_loan_train_processed = pd.read_csv("../data/data_train_processed.csv")
df_loan_test_processed = pd.read_csv("../data/data_test_processed.csv")

In [6]:
df_loan_train_processed.head()

,CODE_GENDER,DAYS_EMPLOYED,FLAG_OWN_CAR,DAYS_ID_PUBLISH,AMT_GOODS_PRICE,EXT_SOURCE_3,DAYS_REGISTRATION,NAME_INCOME_TYPE,FLAG_DOCUMENT_18,AMT_ANNUITY,AMT_REQ_CREDIT_BUREAU_QRT,DAYS_BIRTH,ORGANIZATION_TYPE,NAME_EDUCATION_TYPE,OCCUPATION_TYPE,EXT_SOURCE_1,DAYS_LAST_PHONE_CHANGE,AMT_CREDIT,FLAG_DOCUMENT_3,DEF_30_CNT_SOCIAL_CIRCLE,NAME_FAMILY_STATUS,REGION_RATING_CLIENT_W_CITY,EXT_SOURCE_2,NAME_CONTRACT_TYPE,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,REG_CITY_NOT_LIVE_CITY,TARGET
0,0,365243.0,1,-4043.0,1350000.0,0.617826,-12769.0,0.054666,0,37125.0,0.0,-22960.0,0.054791,0.089190,0.065155,0.506530,-2110.0,1350000.0,1,0.0,0.075539,3,0.168928,0,0,0,0,0
1,0,-2271.0,1,-1325.0,270000.0,0.502878,-4825.0,0.095780,0,13500.0,0.0,-17801.0,0.081121,0.089190,0.105720,0.506530,-1519.0,270000.0,0,0.0,0.075539,2,0.685520,1,0,0,0,0
2,0,365243.0,1,-3980.0,603000.0,0.493863,-280.0,0.054666,0,38844.0,2.0,-22080.0,0.054791,0.053950,0.065155,0.624748,-836.0,758214.0,1,0.0,0.099723,2,0.516795,0,0,0,1,0
3,1,-2128.0,0,-2628.0,405000.0,0.621226,-3201.0,0.074904,0,20250.0,0.0,-19070.0,0.078670,0.089190,0.062960,0.506530,-1075.0,405000.0,0,0.0,0.075539,3,0.693595,1,0,0,0,0
4,1,365243.0,0,-4154.0,180000.0,0.535276,-4145.0,0.054666,0,11506.5,0.0,-24194.0,0.054791,0.111291,0.065155,0.506530,0.0,203760.0,0,0.0,0.075539,2,0.263144,0,1,0,0,0


### Separar en Train y Test

In [7]:
#### Train-Test split
y_train = df_loan_train_processed['TARGET']
X_train = df_loan_train_processed.drop('TARGET', axis=1)
y_test = df_loan_test_processed['TARGET']
X_test = df_loan_test_processed.drop('TARGET', axis=1)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((246004, 27), (246004,), (61503, 27), (61503,))

In [8]:
print(y_train.value_counts(normalize=True) * 100)
print(y_test.value_counts(normalize=True) * 100)

TARGET
0    91.926961
1     8.073039
Name: proportion, dtype: float64
TARGET
0    91.927223
1     8.072777
Name: proportion, dtype: float64


### Validation split

In [9]:
# Divide train again into train and validation subsets
X_train_new, X_val, y_train_new, y_val = train_test_split(X_train, y_train
                                                  ,stratify=df_loan_train_processed['TARGET']
                                                  ,test_size=0.2
                                                  ,random_state=seed)

## **Oversampling & Undersampling**


Esta estrategia mixta tiene como objetivo equilibrar la distribución de las clases en nuestra variable objetivo. Al combinar técnicas de oversampling y undersampling, buscamos reducir la clase mayoritaria mientras aumentamos la clase minoritaria de forma proporcional. El propósito es mantener un número total de registros similar al del dataset original de entrenamiento, pero con una distribución más equilibrada de las clases.

Con este enfoque, los modelos se entrenarán con un conjunto de datos balanceado, lo que permitirá que ambos grupos sean representados de manera equitativa. La combinación de oversampling y undersampling permite que el modelo aprenda de manera más efectiva de la clase minoritaria, evitando el sesgo hacia la clase mayoritaria y mejorando la precisión del modelo en general.

In [10]:
# Under and Over Sampling
overs = SMOTE(sampling_strategy=0.1, random_state=seed)
unders = RandomUnderSampler(sampling_strategy=0.2, random_state=seed)

# Create pipeline
steps = [('o', overs), ('u', unders)]
pipeline = Pipeline(steps=steps)

# Resample
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train_new, y_train_new)


print(f"Original dataset shape: {y_train.value_counts(normalize=True)}")
print(f"Resampled dataset shape: {y_train_resampled.value_counts(normalize=True)}")

Original dataset shape: TARGET
0    0.91927
1    0.08073
Name: proportion, dtype: float64
Resampled dataset shape: TARGET
0    0.833333
1    0.166667
Name: proportion, dtype: float64


Después de aplicar la técnica de resampling, el nuevo conjunto de datos está más balanceado, y por lo tanto alcanzando un 16.67% de instancias positivas en la variable objetivo. Este ajuste mejora la representación de la clase minoritaria, lo que permite que los modelos se entrenen de manera más equilibrada, manteniendo la consistencia con el tamaño original del conjunto de datos.

## **SELECCIÓN DE MODELOS**

En esta sección nos enfocamos en la selección y evaluación de modelos. Comenzamos por comparar diversos modelos de clasificación utilizando sus configuraciones predeterminadas, sin ningún ajuste previo, para obtener una visión general de su rendimiento inicial. Este enfoque nos permite identificar el modelo con el mejor desempeño en términos de precisión y capacidad de generalización sobre el conjunto de datos.

Una vez seleccionado el modelo más prometedor, se procederá a optimizar sus hiperparámetros. Para ello, utilizaremos técnicas de búsqueda de hiperparámetros, como la búsqueda en rejilla o la búsqueda aleatoria, combinadas con validación cruzada para asegurar que el modelo seleccionado no sólo sea eficaz, sino también robusto y adaptado a las características específicas del conjunto de datos.


### **Modelo base**

Como modelo base vamos a tomar el modelo más elemental posible, que sería predecir el valor de la clase predominante en todos los casos, lo que en este caso de estudio significarñia predecir que no va a haber impago en ningún caso (0 en la variable ojetivo).

In [12]:
y_pred_base = f.y_pred_modelo_base(y_train_new, X_val)

In [15]:
f.all_metrics(y_val, y_pred_base)

Accuracy: 0.91927
Balanced Accuracy: 0.50000
F2 Score: 0.00000
F1 Score: 0.00000
Precision: 0.00000
Recall: 0.00000

Confusion Matrix:
[[45229     0]
 [ 3972     0]]


Este modelo, aunque sea básico, nos sirve como punto de partida para ver qué tan bien funcionan los modelos más avanzados.

### **1. Comparación de modelos**

#### Regresión logística

In [16]:
lr = LogisticRegression(C=1)
lr.fit(X_train_new, y_train_new)

y_pred_lr = lr.predict(X_val)

f.all_metrics(y_val, y_pred_lr)

Accuracy: 0.91927
Balanced Accuracy: 0.50000
F2 Score: 0.00000
F1 Score: 0.00000
Precision: 0.00000
Recall: 0.00000

Confusion Matrix:
[[45229     0]
 [ 3972     0]]


#### Support Vector Machine

In [17]:
X_train_subsampled = X_train_new.sample(40000, random_state=seed)
y_train_subsampled = y_train_new.sample(40000, random_state=seed)
X_val_subsampled = X_val.sample(40000, random_state=seed)
y_val_subsampled = y_val.sample(40000, random_state=seed)

X_train_subsampled.shape, y_train_subsampled.shape, X_val_subsampled.shape, y_val_subsampled.shape

((40000, 27), (40000,), (40000, 27), (40000,))

In [ ]:
svc = SVC(kernel="rbf", degree=3) # utilizo los valores por defecto.
svc.fit(X_train_subsampled, y_train_subsampled)

y_pred_svc = svc.predict(X_val_subsampled)
f_aux.summarize_metrics(y_val_subsampled, y_pred_svc)

### Pipeline de selección de modelos

In [19]:
classifiers = [
    DummyClassifier() # modelo base de SKLearn
    ,GaussianNB() # Clasificador de NaiveBayes
    ,DecisionTreeClassifier() # Árbol de decisión simple
    ,GradientBoostingClassifier() # Modelo boosting
    ,RandomForestClassifier() # Random Forest en representación de modelos de Bagging
    ,XGBClassifier() # Modelo boosting
    ,lgb.LGBMClassifier() # Modelo boosting
    ,AdaBoostClassifier() # Modelo boosting
    ]

for classifier in classifiers:
    pipe = Pipeline(steps=[('classifier', classifier)])
    pipe.fit(X_train_new, y_train_new)   
    y_pred_pipe = pipe.predict(X_val)
    print(f'\033[1m{classifier}\033[0m')
    f.all_metrics(y_val, y_pred_pipe)
    print("__________________________________________________________________________________________")

DummyClassifier()
Accuracy: 0.91927
Balanced Accuracy: 0.50000
F2 Score: 0.00000
F1 Score: 0.00000
Precision: 0.00000
Recall: 0.00000

Confusion Matrix:
[[45229     0]
 [ 3972     0]]
__________________________________________________________________________________________
GaussianNB()
Accuracy: 0.91927
Balanced Accuracy: 0.50000
F2 Score: 0.00000
F1 Score: 0.00000
Precision: 0.00000
Recall: 0.00000

Confusion Matrix:
[[45229     0]
 [ 3972     0]]
__________________________________________________________________________________________
DecisionTreeClassifier()
Accuracy: 0.85466
Balanced Accuracy: 0.54478
F2 Score: 0.17010
F1 Score: 0.16294
Precision: 0.15226
Recall: 0.17523

Confusion Matrix:
[[41354  3875]
 [ 3276   696]]
__________________________________________________________________________________________
GradientBoostingClassifier()
Accuracy: 0.91949
Balanced Accuracy: 0.50552
F2 Score: 0.01471
F1 Score: 0.02318
Precision: 0.56627
Recall: 0.01183

Confusion Matrix:
[[45193  

Después de evaluar todos los modelos, los que han demostrado el mejor rendimiento son el **'DecisionTreeClassifier()** y los modelos de boosting: **'LGBMClassifier()'**, **'XGBClassifier()'**, **'AdaBoostClassifier()'** y **'GradientBoostingClassifier()'**. Estos modelos no solo presentan métricas destacables, sino que también ofrecen una buena capacidad de optimización y han mostrado una gran estabilidad en el entrenamiento. Además, en comparación con otros enfoques más complejos, como los de bagging, estos modelos de boosting y el árbol de decisión sobresalen por su capacidad de ajustarse de manera eficiente a los datos. Si bien no se puede descartar la posibilidad de mejorar aún más con la optimización de hiperparámetros, estos modelos son los que, por ahora, mejor equilibran rendimiento y tiempo de computación. Sin embargo, seguiremos evaluando su desempeño para confirmar que mantienen su efectividad en diferentes configuraciones.

### **2. Optimización de hiperparámetros**

#### LightGBM

In [20]:
f2_scorer = make_scorer(fbeta_score, beta=2)

lgbm = lgb.LGBMClassifier(verbosity=-1, is_unbalance=True, objetive='binary')
param_grid = {
    'n_estimators ': np.random.randint(100,1000,10) 
    ,'max_depth' : np.random.randint(3,10,10)
    ,'num_leaves' : np.random.randint(round(2**3/1.5),round(2**10/1.5),30) # (num_leaves = (2^max_depth/1.5))
    ,'min_data_in_leaf' : np.random.randint(1000,2000,10)
    ,'learning_rate': np.random.uniform(0.01, 0.05,10)
    ,'subsample': np.random.uniform(0.8,1,10)
    ,'colsample_bytree': np.random.uniform(0.8,1,10)
    ,'min_child_samples': np.random.randint(20,1000,30)
}

scoring = {'f2':f2_scorer,'f1':'f1','accuracy':'accuracy', 'balanced_accuracy':'balanced_accuracy', 'precision':'precision', 'recall':'recall','roc_auc':'roc_auc'}

CV_rs = RandomizedSearchCV(lgbm, param_grid, n_iter=50, cv=20, random_state=seed, n_jobs=-1, scoring=scoring, refit='f2')

CV_rs.fit(X_train_new, y_train_new)  
print(CV_rs.best_params_)    
print(CV_rs.best_score_)

{'subsample': 0.9749394912418917, 'num_leaves': 209, 'n_estimators ': 188, 'min_data_in_leaf': 1454, 'min_child_samples': 528, 'max_depth': 8, 'learning_rate': 0.04053035597617399, 'colsample_bytree': 0.945656458361092}
0.41724860713847994


In [21]:
CV_rs.best_estimator_

LGBMClassifier(colsample_bytree=0.945656458361092, is_unbalance=True,
               learning_rate=0.04053035597617399, max_depth=8,
               min_child_samples=528, min_data_in_leaf=1454, n_estimators =188,
               num_leaves=209, objetive='binary', subsample=0.9749394912418917,
               verbosity=-1)

In [22]:
CV_rs.best_params_

{'subsample': 0.9749394912418917,
 'num_leaves': 209,
 'n_estimators ': 188,
 'min_data_in_leaf': 1454,
 'min_child_samples': 528,
 'max_depth': 8,
 'learning_rate': 0.04053035597617399,
 'colsample_bytree': 0.945656458361092}

In [23]:
CV_rs.best_score_

0.41724860713847994

#### XGBoost

In [24]:
f2_scorer = make_scorer(fbeta_score, beta=2)

xgb = XGBClassifier(verbosity=0, objective='binary:logistic', use_label_encoder=False)

param_grid = {
    'n_estimators': np.random.randint(100, 1000, 10),
    'max_depth': np.random.randint(3, 10, 10),
    'learning_rate': np.random.uniform(0.01, 0.05, 10),
    'subsample': np.random.uniform(0.8, 1, 30),
    'colsample_bytree': np.random.uniform(0.8, 1, 10),
    'min_child_weight': np.random.randint(1, 10, 10),
    'gamma': np.random.uniform(0, 0.5, 10),
    'max_leaves': np.random.randint(1, 10, 10)
}

scoring = {'f2': f2_scorer, 'f1': 'f1', 'accuracy': 'accuracy', 'balanced_accuracy': 'balanced_accuracy', 'precision': 'precision', 'recall': 'recall', 'roc_auc': 'roc_auc'}

CV_rs2 = RandomizedSearchCV(xgb, param_grid, n_iter=50, cv=10, random_state=seed, n_jobs=-1, scoring=scoring, refit='f2')

CV_rs2.fit(X_train_new, y_train_new)
print(CV_rs2.best_params_)
print(CV_rs2.best_score_)

{'subsample': 0.8450517312116986, 'n_estimators': 865, 'min_child_weight': 4, 'max_leaves': 5, 'max_depth': 9, 'learning_rate': 0.04122879257345018, 'gamma': 0.22098017584158963, 'colsample_bytree': 0.9230232975495047}
0.02456069379092725


In [25]:
CV_rs2.best_estimator_

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9230232975495047, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.22098017584158963,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04122879257345018,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=5,
              min_child_weight=4, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=865, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [26]:
CV_rs2.best_params_

{'subsample': 0.8450517312116986,
 'n_estimators': 865,
 'min_child_weight': 4,
 'max_leaves': 5,
 'max_depth': 9,
 'learning_rate': 0.04122879257345018,
 'gamma': 0.22098017584158963,
 'colsample_bytree': 0.9230232975495047}

In [27]:
CV_rs2.best_score_

0.02456069379092725

### **Exportación de datos y modelos**

In [28]:
pd.concat([X_train_new,y_train_new], axis=1).to_csv('../data/df_train_resampled.csv', index=False)

In [29]:
dump(CV_rs, '../models/CV_lgbm.joblib')

['../models/CV_lgbm.joblib']

In [30]:
dump(CV_rs2, '../models/CV_xgboost.joblib')

['../models/CV_xgboost.joblib']